<p align="right" width="100%"><img width="200px" height="auto" src="../Admin/eth_logo_kurz_pos.png">

# Mobile Computing

## Exercise: Audio Communication
### Prerequisites

This Jupyter Notebook has been tested with Visual Studio Code, running in a local Python environment.

In [ ]:
%pip install --quiet matplotlib scipy soundfile numpy
from matplotlib import mlab
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
from scipy.signal import get_window
from scipy.fft import fft

### 1. Sound Visualization
#### 1.1 Loading and preprocessing the audio file
The first step is to load the audio signal and normalize it.

In [ ]:
def load_audio(audio_file, time_interval):
    try:
        file_info = sf.info(audio_file)
    except RuntimeError as e:
        raise RuntimeError(f"Error reading the audio file: {e}") 
    
    sample_rate = file_info.samplerate
    
    start_sample = int(round(time_interval[0] * sample_rate))
    end_sample = int(round(time_interval[1] * sample_rate))
    start_sample = max(start_sample, 0)
    end_sample = min(end_sample, file_info.frames)
    
    try:
        with sf.SoundFile(audio_file) as f:
            f.seek(start_sample)
            audio_data = f.read(end_sample - start_sample)
    except RuntimeError as e:
        raise RuntimeError(f"Error reading samples from the audio file: {e}")
    
    audio_data = np.array(audio_data)
    
    # Converting to mono if there are multiple channels
    if audio_data.ndim > 1:
        audio_data = audio_data[:, 0]
    
    # Normalizing the audio signal
    audio_max = np.max(np.abs(audio_data))
    if audio_max != 0:
        audio_data = audio_data / audio_max
    
    audio_length = len(audio_data)
    time_vector = np.arange(audio_length) / sample_rate
    
    return audio_data, sample_rate, time_vector, audio_length

With the `load_audio` function available, we can now select the audio file that we wish to visualize. 

Additionally, we also define the desired time interval for the visualization.

In [ ]:
audio_file = 'Sounds/MobileComp2.wav'
time_interval = [1, 5]
audio_data, sample_rate, time_vector, audio_length = load_audio(audio_file, time_interval)

#### 1.2 Time Domain Plot
The time domain plot displays how a signal's amplitude changes over time.

In [ ]:
def plot_time():
    plt.figure(figsize=(10, 4))
    plt.plot(time_vector, audio_data, linewidth=0.5)
    plt.xlim([0, time_vector[-1]])
    plt.ylim([-1.1*max(abs(audio_data)), 1.1*max(abs(audio_data))])
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.xlabel('Time [s]', fontsize=12)
    plt.ylabel('Normalized Amplitude', fontsize=12)
    plt.title(f'{audio_file} - Time Domain', fontsize=14)
    plt.tight_layout()
    plt.show()


plot_time()

#### 1.3 Frequency Domain Plot
The frequency domain plot illustrates how a signal's energy is distributed across different frequency components. For this, we first translate the signal by using scipy's `fft` function.

In [ ]:
def plot_frequency():
    window = get_window('hann', audio_length)
    window_avg = np.sum(window) / audio_length
    magnitude = np.abs(fft(audio_data * window)) # computing the FFT on the windowed audio signal
    pos_freq = magnitude[:audio_length // 2] # extract first half (positive frequencies)
    pos_freq = pos_freq / (audio_length / 2) # normalize
    if len(pos_freq) > 0:
        pos_freq[0] = pos_freq[0] / 2  # Correct the DC component
    pos_freq = pos_freq / window_avg # compensate for windowing

    freq_axis = np.linspace(0, sample_rate / 2, len(pos_freq)) # create frequency axis

    pos_freq = np.maximum(pos_freq, 1e-10) # Prevent log of 0

    plt.figure(figsize=(10, 4))
    plt.plot(freq_axis / 1000, 20 * np.log10(pos_freq), linewidth=0.5)
    plt.xlim([freq_axis[0] / 1000, freq_axis[-1] / 1000])
    plt.grid(True)
    plt.xlabel('Frequency [kHz]', fontsize=12)
    plt.ylabel('Amplitude [dBV]', fontsize=12)
    plt.title(f'{audio_file} - Frequency Domain', fontsize=14)
    plt.tight_layout()
    plt.show()


plot_frequency()

#### 1.4 Spectogram
The spectogram displays how the frequency content of a signal changes over time.

It effectively combines the time and frequency domain information.

In [ ]:
def plot_spectogram():
    FFT_win_size = 256
    overlaps = int(FFT_win_size * 0.97)
    window = get_window('hann', FFT_win_size)

    spectral_power, freq, time_bins = mlab.specgram(
        audio_data,
        NFFT=FFT_win_size,
        Fs=sample_rate,
        window=window,
        noverlap=overlaps,
        scale_by_freq=False,
        mode='psd'
    )

    spectral_power_dB = 10 * np.log10(spectral_power + 1e-10) # power to decibels

    vmax = spectral_power_dB.max()
    vmin = vmax - 100  # 100dB dynamic range

    plt.figure(figsize=(10, 4))
    plt.pcolormesh(time_bins, freq / 1000, spectral_power_dB, shading='gouraud', vmin=vmin, vmax=vmax, cmap='plasma')
    plt.colorbar(label='Magnitude [dB]')
    plt.xlabel('Time [s]', fontsize=12)
    plt.ylabel('Frequency [kHz]', fontsize=12)
    plt.title(f'{audio_file} - Spectrogram', fontsize=14)
    plt.ylim(0, sample_rate / 2000)
    plt.tight_layout()
    plt.show()


plot_spectogram()

### 2. Audio Steganography
#### 2.1. Prerequisites

Audio Steganography is the practice of embedding information within audio files.

While the error rate is often the most important metric for evaluating the success of steganography, it is equally important that the concealed data remains undetectable to unintended listeners.

To start off, we first import and install the required packages.

**NOTE**: Some of the subsequent sections may not function properly without FFMPEG.

In [ ]:
%pip install --quiet ipython numpy scipy matplotlib
import os
from AudioSteganography import *
from IPython.display import Audio, display # the audio file display might not work in all IDEs

if not os.path.exists('Out'):
    os.makedirs('Out')

Before we start, we define the audio file for embedding, specify the output file name, and set the message to embed.

In [ ]:
input_filename = "Sounds/MobileComp2.wav"
output_filename = "Out/stego_MobileComp2.wav"
message_filename = "Messages/manimatter.txt"

In [ ]:
with open(message_filename, 'r') as file:
    message = file.read().replace('\n', ' ')

#### 2.2. Encoding
As with any file, we first verify if the audio file is even large enough to hold the text.

In [ ]:
message_length = len(message) * 8
max_capacity = calculate_max_message_length(input_filename)
print(f"Maximum message length in bytes: {max_capacity}")
print(f"Message size in bytes: {message_length}")

If the message length is less than or equal to the maximum capacity, we proceed with embedding the message into the audio using phase coding.

In [ ]:
embed_message(input_filename, output_filename, message)
print(f'Embedded message:\n{message}')

display(Audio(filename=input_filename, autoplay=False))

#### 2.3. Decoding
Now, we use the same algorithm in reverse to extract the message.

In [ ]:
extracted_message = extract_message(output_filename, 8 * len(message))
print(f'Extracted Message:\n{extracted_message}')

display(Audio(filename=output_filename, autoplay=False))

#### 2.4. Accuracy
Finally, we compare the original message to the extracted message to calculate the bit error rate.


In [ ]:
ber, _ = calculate_accuracy(message, extracted_message)
print(f'Bit Error Rate (BER): {ber:.2%}')

#### 2.5. Visualization
Just like we did in section 1, we can also visualize the changes we introduced with our embedding.

In [ ]:
audio_file = 'Out/stego_MobileComp2.wav'
time_interval = [1, 5]
audio_data, sample_rate, time_vector, audio_length = load_audio(audio_file, time_interval)

plot_spectogram()

### 3. Further Experiments
Now that you have seen how to do it, it is your turn to use your own audio file.
Below, you find a snippet that converts your audio file to the right format and right channel configuration for this notebook.

In [ ]:
file_to_convert = "AudioFile.xyz"
converted_file_name = "AudioFile.wav"

In [ ]:
%pip install --quiet pydub
from pydub import AudioSegment

In [ ]:
sound = AudioSegment.from_file(file_to_convert)
    
if sound.channels > 1:
    channels = sound.split_to_mono()
    mono_sound = channels[0].overlay(channels[1])
else:
    mono_sound = sound

mono_sound.export(converted_file_name, format="wav")
print(f"File converted to mono WAV and saved as {converted_file_name}")

<p align="center" width="100%">═════════════════════════════════<br>
 Stefan Mangold (<a href="mailto:stefan.mangold@inf.ethz.ch">stefan.mangold@inf.ethz.ch</a>)<br><img width="200px" height="auto" src="../Admin/eth_logo_kurz_pos.png"></p>